In [15]:
# importing necessary libraries
import sys
import numpy
import pandas
import random
import logging
import warnings
import threading
import mlflow.sklearn
from itertools import cycle, islice
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [16]:
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

In [17]:
mlflow.set_tracking_uri('http://localhost:5000')
mlflow.set_experiment(experiment_name='Wine Regression')
tags = {
    "team": "Cold Red Wine",
    "dataset": "Wine",
    "release.version": "2.2.2"
}

2023/06/11 02:46:37 INFO mlflow.tracking.fluent: Experiment with name 'Wine Regression' does not exist. Creating a new experiment.


In [18]:
def eval_metrices(actual, pred):
    root_mean_square_error = numpy.sqrt(mean_squared_error(actual, pred))
    mean_abs_err = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return root_mean_square_error, mean_abs_err, r2

In [19]:
thread_lock = threading.Lock()

def mlflow_func(alpha, l1_ratio, train_x, train_y, test_x, test_y, run_name):
    with mlflow.start_run(run_name=run_name):
        mlflow.set_tags(tags)
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrices(test_y, predicted_qualities)
        
        with thread_lock:
            print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
            print("  RMSE: %s" % rmse)
            print("  MAE: %s" % mae)
            print("  R2: %s" % r2)
            mlflow.log_param("alpha", alpha)
            mlflow.log_param("l1_ratio", l1_ratio)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)
            mlflow.sklearn.log_model(lr, "model")
            mlflow.log_artifact(local_path='./demo.ipynb', artifact_path='code')

In [20]:
if __name__ == "__main__":
    warnings.filterwarnings("ignore")
    # Read the wine-quality csv file
    run_nams = ['Apples',        'Apricots',     'Asparagus', 
                'Avocados',      'Cherries',     'Basil', 
                'Beets',         'Blackberries', 'Blueberries', 
                'Boysenberries', 'Broccoli',     'Cabbage', 
                'Carrots',       'Collards',     'Corn', 
                'Cucumber',      'Eggplants']

    csv_file_path = "./winequality-red.csv"
    try:
        data = pandas.read_csv(csv_file_path, sep=";")
    except Exception as e:
        logger.exception("Unable to read the CSV file. Error: %s", e)
    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)
    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

    iterations = 16
    # Generate unique run names for each thread
    unique_run_names = list(islice(cycle(run_nams), iterations))

    # Execute threads
    threads = []
    for i, run_name in enumerate(unique_run_names):
        alpha = round(random.random(), 1) # Default value
        if len(sys.argv) > 1:
            try:
                alpha = float(sys.argv[1])
            except ValueError:
                print("Invalid float value provided. Using default value.")
        l1_ratio = round(random.random(), 1)  # Default value
        if len(sys.argv) > 2:
            try:
                l1_ratio = float(sys.argv[2])
            except ValueError:
                print("Invalid float value provided for l1_ratio. Using default value.")
        thread = threading.Thread(
            target=mlflow_func,
            args=(alpha, l1_ratio, train_x, train_y, test_x, test_y, unique_run_names[i]),
        )
        thread.start()
        threads.append(thread)

    # Wait for all threads to finish
    for thread in threads:
        thread.join()

Invalid float value provided. Using default value.
Invalid float value provided for l1_ratio. Using default value.
Invalid float value provided. Using default value.
Invalid float value provided for l1_ratio. Using default value.
Invalid float value provided. Using default value.
Invalid float value provided for l1_ratio. Using default value.
Invalid float value provided. Using default value.
Invalid float value provided for l1_ratio. Using default value.
Invalid float value provided. Using default value.
Invalid float value provided for l1_ratio. Using default value.
Invalid float value provided. Using default value.
Invalid float value provided for l1_ratio. Using default value.
Invalid float value provided. Using default value.
Invalid float value provided for l1_ratio. Using default value.
Invalid float value provided. Using default value.
Invalid float value provided for l1_ratio. Using default value.
Invalid float value provided. Using default value.
Invalid float value provided 

Elasticnet model (alpha=1.000000, l1_ratio=0.400000):
  RMSE: 0.7808284166101064
  MAE: 0.6401768015415683
  R2: 0.032607596369109815
Elasticnet model (alpha=0.400000, l1_ratio=0.100000):
  RMSE: 0.6869217620764348
  MAE: 0.5526752891439501
  R2: 0.2513031549869488
Elasticnet model (alpha=0.700000, l1_ratio=0.600000):
  RMSE: 0.7809623878645134
  MAE: 0.6406673712740647
  R2: 0.03227560565409382
Elasticnet model (alpha=0.800000, l1_ratio=0.700000):
  RMSE: 0.7822416144063625
  MAE: 0.6440972807823061
  R2: 0.029102718893963764
Elasticnet model (alpha=0.100000, l1_ratio=0.900000):
  RMSE: 0.6829684498754435
  MAE: 0.5504824282142496
  R2: 0.2598960266955982
Elasticnet model (alpha=0.600000, l1_ratio=0.200000):
  RMSE: 0.709630791027218
  MAE: 0.5725620580804106
  R2: 0.20098238249262268
Elasticnet model (alpha=0.100000, l1_ratio=0.100000):
  RMSE: 0.6608286137355012
  MAE: 0.5288184146140058
  R2: 0.30710228109111104
Elasticnet model (alpha=0.800000, l1_ratio=0.100000):
  RMSE: 0.707693

Exception in thread Thread-67 (mlflow_func):
Traceback (most recent call last):
  File "c:\Users\User\Desktop\mlops-project\Lib\site-packages\mlflow\tracking\_tracking_service\client.py", line 297, in log_param
    self.store.log_param(run_id, param)
  File "c:\Users\User\Desktop\mlops-project\Lib\site-packages\mlflow\store\tracking\rest_store.py", line 211, in log_param
    self._call_endpoint(LogParam, req_body)
  File "c:\Users\User\Desktop\mlops-project\Lib\site-packages\mlflow\store\tracking\rest_store.py", line 59, in _call_endpoint
    return call_endpoint(self.get_host_creds(), endpoint, method, json_body, response_proto)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\User\Desktop\mlops-project\Lib\site-packages\mlflow\utils\rest_utils.py", line 201, in call_endpoint
    response = verify_rest_response(response, endpoint)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\User\Desktop\mlops-pro

Elasticnet model (alpha=0.400000, l1_ratio=1.000000):
  RMSE: 0.780815010608434
  MAE: 0.6401160899052867
  R2: 0.032640814301690635


In [21]:
import mlflow

runs = mlflow.search_runs()
run_ids = runs['run_id'].tolist()
print(run_ids)

['f081d2ce09fc45cd98ea3d7f62bec2f7', '381bd3a676f24db097a8fa352d85aaf0', 'b94c9bb5a44145c4bf8b6c17ea8dc138', '981cddb036314e178d04a773d68e6a4c', '2643bb487293436e863b808add548cb8', 'f73dfad5dfb841159d35ff6a7d9880b9', '227aed7078a345a6821b3b0085375ea1', '5dbc94a062dc47ae9f6c967df9107713', 'faf672c3fcec4ab69b2f1c5ee7a2790c', '768acdec8439496e93214c615343c016', 'e3b78ced3cf84d379013d69e45bcf68d', 'dd59c920a9394f39b49ec6e1a37677f1', 'bdf50f5b3d4d4762a306d5733ed9ba26', 'ee4a1abb85d04c719dc6f5e35b8eedfe']


In [22]:
print(runs)

                              run_id       experiment_id    status  \
0   f081d2ce09fc45cd98ea3d7f62bec2f7  315903361643259112  FINISHED   
1   381bd3a676f24db097a8fa352d85aaf0  315903361643259112  FINISHED   
2   b94c9bb5a44145c4bf8b6c17ea8dc138  315903361643259112  FINISHED   
3   981cddb036314e178d04a773d68e6a4c  315903361643259112  FINISHED   
4   2643bb487293436e863b808add548cb8  315903361643259112  FINISHED   
5   f73dfad5dfb841159d35ff6a7d9880b9  315903361643259112  FINISHED   
6   227aed7078a345a6821b3b0085375ea1  315903361643259112  FINISHED   
7   5dbc94a062dc47ae9f6c967df9107713  315903361643259112  FINISHED   
8   faf672c3fcec4ab69b2f1c5ee7a2790c  315903361643259112  FINISHED   
9   768acdec8439496e93214c615343c016  315903361643259112  FINISHED   
10  e3b78ced3cf84d379013d69e45bcf68d  315903361643259112  FINISHED   
11  dd59c920a9394f39b49ec6e1a37677f1  315903361643259112  FINISHED   
12  bdf50f5b3d4d4762a306d5733ed9ba26  315903361643259112  FINISHED   
13  ee4a1abb85d04c71